In [12]:
import pandas as  pd
data = pd.read_csv("fraud test.csv")

In [13]:
pip install ace_tools

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Drop the 'Unnamed: 0' column
cleaned_data = data.drop(columns=['Unnamed: 0'])

# Display the first few rows of the cleaned dataset
print(cleaned_data)


       trans_date_trans_time        cc_num  \
0           21/06/2020 12:14  2.291160e+15   
1           21/06/2020 12:14  3.573030e+15   
2           21/06/2020 12:14  3.598220e+15   
3           21/06/2020 12:15  3.591920e+15   
4           21/06/2020 12:15  3.526830e+15   
...                      ...           ...   
555714      31/12/2020 23:59  3.056060e+13   
555715      31/12/2020 23:59  3.556610e+15   
555716      31/12/2020 23:59  6.011720e+15   
555717      31/12/2020 23:59  4.079770e+12   
555718      31/12/2020 23:59  4.170690e+15   

                                    merchant        category     amt    first  \
0                      fraud_Kirlin and Sons   personal_care    2.86     Jeff   
1                       fraud_Sporer-Keebler   personal_care   29.84   Joanne   
2       fraud_Swaniawski, Nitzsche and Welch  health_fitness   41.28   Ashley   
3                          fraud_Haley Group        misc_pos   60.05    Brian   
4                      fraud_Johnston-Casp

In [21]:
# Split the dataset into a train and test set with 4000 for training and 2000 for testing
train_data = cleaned_data.sample(n=185239, random_state=42)
test_data = cleaned_data.drop(train_data.index).sample(n=370480, random_state=42)

# Show the sizes of the datasets to verify the split
train_data_size = train_data.shape[0]
test_data_size = test_data.shape[0]

(train_data_size, test_data_size)


(185239, 370480)

In [22]:
train_data.dtypes

trans_date_trans_time     object
cc_num                   float64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Select object type columns for NLP processing
text_columns = ['merchant', 'category', 'first', 'last', 'gender', 'street', 
                'city', 'state', 'job', 'trans_num', 'dob']

# Combine the text columns into a single dataset
X_train_text = train_data[text_columns]

# Target variable (is_fraud)
y_train = train_data['is_fraud']

# Preprocessing: Use OneHotEncoder for these columns (can also try TF-IDF for certain columns)
preprocessor = ColumnTransformer(
    transformers=[
        ('text', OneHotEncoder(handle_unknown='ignore'), text_columns)
    ]
)

# Create a pipeline with preprocessing and model (RandomForest for example)
model = make_pipeline(preprocessor, RandomForestClassifier(random_state=42))

# Train the model
model.fit(X_train_text, y_train)

# Predict using the model (self-evaluation on train set)
y_pred = model.predict(X_train_text)

# Output classification report to check the relationship between object data and 'is_fraud'
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    184522
           1       1.00      1.00      1.00       717

    accuracy                           1.00    185239
   macro avg       1.00      1.00      1.00    185239
weighted avg       1.00      1.00      1.00    185239

